In [25]:
import requests
import pandas as pd
import requests_cache
from tqdm import tqdm
import grequests

requests_cache.install_cache('demo_cache', allowable_codes=(200,400,404,500))

In [4]:
df = pd.read_json("https://gnps-external.ucsd.edu/gnpslibraryjson")

In [27]:
all_smiles_url = []
# Getting all structures
for index_label, record in tqdm(df.iterrows()):
    smiles = record["Smiles"]
    if len(smiles) > 10:
        url = "https://gnps-structure.ucsd.edu/classyfire?smiles={}".format(smiles)
        all_smiles_url.append(url)
        
rs = (grequests.get(u) for u in all_smiles_url)
requests_returned = grequests.map(rs)

155502it [00:14, 11092.71it/s]


In [38]:
record_list = df.to_dict(orient="records")

for record in tqdm(record_list):
    smiles = record["Smiles"]
    if len(smiles) > 10:
        url = "https://gnps-structure.ucsd.edu/classyfire?smiles={}".format(smiles)
        try:
            r = requests.get(url)
            record["class"] = r.json()["class"]["name"]
        except KeyboardInterrupt:
            raise
        except:
            pass
            
df = pd.DataFrame(record_list)

100%|██████████| 155502/155502 [01:23<00:00, 1854.09it/s]


In [45]:
filtered_df = df[["class", "Precursor_MZ"]]
filtered_df = filtered_df.dropna(axis=0)

In [47]:
filtered_df.to_csv("class_data.tsv", sep="\t", index=False)

,class,Precursor_MZ
0,Peptidomimetics,981.540
1,Peptidomimetics,940.250
3,Indoles and derivatives,545.000
5,Peptidomimetics,667.115
7,Macrolactams,1025.610
...,...,...
155497,Naphthofurans,846.445
155498,Naphthofurans,432.240
155499,Naphthofurans,437.198
155500,Naphthofurans,867.375
